<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Figure 1

_Authors: Oriana Chegwidden and Jeremy Freeman_

The methods below conduct the analyses to recreate Figure 1 included the
manuscript <<< insert doi >>>. They draw from a model data produced by running
`../../../scripts/fire.py` to create a the file located at
`az:carbonplan-scratch/data/fire.zarr`.


In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import scipy as sp
from carbonplan_forests import load, setup, plot, fit, utils, prepare, collect
import xarray as xr
from carbonplan_forests.utils import get_store


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from carbonplan_forests import load, setup, plot, fit, utils, prepare, collect
from showit import image
import scipy as sp
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import regionmask as rm
from scipy.stats import binom
import altair as alt

from palettable.colorbrewer.sequential import YlOrRd_9

In [ ]:
coarsen = 4
store = "az"
tlim = ("1984", "2018")

In [ ]:
historical_ds = xr.open_zarr(get_store("carbonplan-scratch", "data/fire.zarr"))

In [ ]:
mask = load.mask(store=store, year=2001)

In [ ]:
nlcd = load.nlcd(store=store, year=2001, coarsen=coarsen, mask=mask)
nftd = load.nftd(store=store, area_threshold=1500, coarsen=coarsen, mask=mask)

In [ ]:
mtbs = load.mtbs(store=store, coarsen=coarsen, tlim=tlim, mask=mask)
# give it the same x/y coords as nftd (and nlcd and climate)
# this re-assignment will help us with some masking lower down
mtbs = mtbs.assign_coords({"x": nftd.x, "y": nftd.y})

In [ ]:
alt.data_transformers.disable_max_rows()

In [ ]:
plot.fire.simple_map(
    mtbs["monthly"].groupby("time.year").sum().mean(dim="year"),
    data2=historical_ds["historical"]
    .groupby("time.year")
    .sum()
    .mean(dim="year"),
    clim=(0.001, 0.01),
    clabel="Probability",
    cmap=["white", "#ea9755"],
)